In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
import librosa
import matplotlib.pyplot as plt
import os
from datetime import datetime
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns',None)

In [2]:
# output_file_date = datetime.now().strftime("%Y-%m-%d %H:%M")
output_file_date = datetime.now().strftime("%Y-%m-%d")
output_file_date

'2023-08-09'

In [3]:
def get_mfcc(wav_file_path):
  y, sr = librosa.load(wav_file_path, offset=10, duration=60)
  mfcc = np.array(librosa.feature.mfcc(y=y, sr=sr))
  return mfcc
    
def get_melspectrogram(wav_file_path):
  y, sr = librosa.load(wav_file_path, offset=10, duration=60)
  melspectrogram = np.array(librosa.feature.melspectrogram(y=y, sr=sr))
  return melspectrogram

def get_chroma_vector(wav_file_path):
  y, sr = librosa.load(wav_file_path)
  chroma = np.array(librosa.feature.chroma_stft(y=y, sr=sr))
  return chroma

def get_tonnetz(wav_file_path):
  y, sr = librosa.load(wav_file_path)
  tonnetz = np.array(librosa.feature.tonnetz(y=y, sr=sr))
  return tonnetz

def get_feature(file_path):
  # Extracting MFCC feature
  mfcc = get_mfcc(file_path)
  mfcc_mean = mfcc.mean(axis=1)
  mfcc_min = mfcc.min(axis=1)
  mfcc_max = mfcc.max(axis=1)
  mfcc_feature = np.concatenate( (mfcc_mean, mfcc_min, mfcc_max) )

  # Extracting Mel Spectrogram feature
  melspectrogram = get_melspectrogram(file_path)
  melspectrogram_mean = melspectrogram.mean(axis=1)
  melspectrogram_min = melspectrogram.min(axis=1)
  melspectrogram_max = melspectrogram.max(axis=1)
  melspectrogram_feature = np.concatenate( (melspectrogram_mean, melspectrogram_min, melspectrogram_max) )

  # Extracting chroma vector feature
  chroma = get_chroma_vector(file_path)
  chroma_mean = chroma.mean(axis=1)
  chroma_min = chroma.min(axis=1)
  chroma_max = chroma.max(axis=1)
  chroma_feature = np.concatenate( (chroma_mean, chroma_min, chroma_max) )

  # Extracting tonnetz feature
  tntz = get_tonnetz(file_path)
  tntz_mean = tntz.mean(axis=1)
  tntz_min = tntz.min(axis=1)
  tntz_max = tntz.max(axis=1)
  tntz_feature = np.concatenate( (tntz_mean, tntz_min, tntz_max) ) 
  
  feature = np.concatenate( (chroma_feature, melspectrogram_feature, mfcc_feature, tntz_feature) )
  return feature

In [4]:
# Need to read in Kaggle data for reverse lookup
# IE: Once we have the song recommendations, we want to look up the information so that we can present that to the user
# kaggle = pd.read_csv("../data/SpotifyFeatures.csv")

In [5]:
data_dir = Path('../data/mp3s/')
output_dir = Path('../data/vectorized_mp3s/')
path_glob = data_dir.rglob('*.mp3')
file_paths = []
for file_path in path_glob:
    file_paths.append(file_path) # creates a list for repeated iteration
    # if this is not done, the .rglob command above has to be repeated to regenerate iterator
len(file_paths) # number of mp3s in directory

11571

In [6]:
# for tuple in enumerate(file_paths):
#     print(tuple) # used to find tracks that fail to convert

In [7]:
# 11572/500

In [8]:
# Had to delete an mp3 as it was only 4 seconds long. 
# I can't explain how that happened, but the code didn't have any issue with the 
# other file paths.

In [9]:
# downloaded_track_ids = []
# for file_path in file_paths:
#     # print(file_path)
#     print(f'{count}. FILE PATH: \n', f'{file_path} \n')
#     path_split = str(file_path).split('/')
#     track_ids.append(path_split[3])

In [10]:
# vectorized_track_ids = []
# path_glob = output_dir.rglob('*.mp3')
# for file_path in path_glob:
#     # print(file_path)
#     # print(f'{count}. FILE PATH: \n', f'{file_path} \n')
#     path_split = str(file_path).split('/')
#     vectorized_track_ids.append(path_split[3])
# len(vectorized_track_ids)

In [11]:
# tracks = []
data_dir = Path('../data/mp3s/')
output_dir = Path('../data/vectorized_mp3s/')
downloaded_path_glob = data_dir.rglob('*.mp3')
output_path_glob = output_dir.rglob('*.parquet')
count = 1
file_paths = [file_path for file_path in downloaded_path_glob]
print('Number of MP3 Files: ', len(file_paths),'\n')
vectorized_track_ids = [file_path.stem for file_path in output_path_glob]
for file_path in file_paths:
    print(f'{count}. FILE PATH: \n', f'{file_path}')
    path_split = str(file_path).split('/')
    track_id = path_split[3]
    if (len(vectorized_track_ids)>0) & (track_id in vectorized_track_ids):
        print(f'{track_id} has already been vectorized...skipping...')
        count+=1
    else:
        track = get_feature(file_path)
        vectorized_df = pd.DataFrame(track).T
        vectorized_df['track_id'] = track_id
        vectorized_df = vectorized_df.set_index(vectorized_df.track_id).drop(columns = 'track_id')
        vectorized_df.columns = vectorized_df.columns.astype(str)
        vectorized_df.to_parquet(f"../data/vectorized_mp3s/{track_id}.parquet")
        # vectorized_df.to_csv(f"../data/vectorized_mp3s/{track_id}.csv")
        count+=1
# vectorized_df = pd.DataFrame(tracks, index = track_ids)
# This implementation also scales everything to 498 dimensions
# This loop should write out every file as it moves through the data.
# Each one will be 500 rows accept the last one which should be 73
# This should create 24 files

Number of MP3 Files:  11571 

1. FILE PATH: 
 ../data/mp3s/1ZB2qWsheGabSEYvBYxjKn/Take on Me/Weezer - Take on Me.mp3
1ZB2qWsheGabSEYvBYxjKn has already been vectorized...skipping...
2. FILE PATH: 
 ../data/mp3s/5V9H9J5GcUGY5ig029g5OU/Shkleepy/Manwolves - Shkleepy.mp3
5V9H9J5GcUGY5ig029g5OU has already been vectorized...skipping...
3. FILE PATH: 
 ../data/mp3s/34FsCOAQ0U99vAh3uoiLmm/Bandana (feat. Young Buck)/Dirty Audio, BL3R, Young Buck - Bandana (feat. Young Buck).mp3
34FsCOAQ0U99vAh3uoiLmm has already been vectorized...skipping...
4. FILE PATH: 
 ../data/mp3s/25mldAmMHYzXhDXCxTpTHy/Chloroform/Phoenix - Chloroform.mp3
25mldAmMHYzXhDXCxTpTHy has already been vectorized...skipping...
5. FILE PATH: 
 ../data/mp3s/1YaOBTTdptDf4vYKpFy56T/Mawaranai Toe Shoes/majiko - Mawaranai Toe Shoes.mp3
1YaOBTTdptDf4vYKpFy56T has already been vectorized...skipping...
6. FILE PATH: 
 ../data/mp3s/2RbDFTlqdIdiZwO4GaTxi2/moonwalking/Good Scott - moonwalking.mp3
2RbDFTlqdIdiZwO4GaTxi2 has already been vect

/Users/vii/anaconda3/envs/brainstation_capstone/lib/python3.8/site-packages/librosa/core/pitch.py:102: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(


10160. FILE PATH: 
 ../data/mp3s/2a7z0RvsjwSMWEPwIWO5z2/La Ultima Vez/Feid - La Ultima Vez.mp3
10161. FILE PATH: 
 ../data/mp3s/7os1J5P6JqIxKSxq71QuQC/Hero (feat. Christina Perri) - Deep Mix/Cash Cash, Christina Perri - Hero (feat. Christina Perri) - Deep Mix.mp3
10162. FILE PATH: 
 ../data/mp3s/4qZwVr553XcQNdeqjueeE3/Roses Are Red - Original Version/Aqua - Roses Are Red - Original Version.mp3
10163. FILE PATH: 
 ../data/mp3s/48DPyFzSUNGAYzXM9Zj4Jd/La rondine Chi il bel sogno di Doretta/Giacomo Puccini, Luba Orgonasova, Symfonický orchester Slovenského rozhlasu, Will Humburg - La rondine# Chi il bel sogno di Doretta.mp3
10164. FILE PATH: 
 ../data/mp3s/4xLRNLT9MkHFNpbxuvHmMC/Divertimento No. 10 in F Major, K. 247 III. Menuetto - Trio/Wolfgang Amadeus Mozart, Camerata Salzburg, Sándor Végh - Divertimento No. 10 in F Major, K. 247# III. Menuetto - Trio.mp3
10165. FILE PATH: 
 ../data/mp3s/7pUqeYHnZI8RdhGS2BkAN5/Steel On Steel/Johan Söderqvist, Patrik Andrén - Steel On Steel.mp3
10166. FI

/Users/vii/anaconda3/envs/brainstation_capstone/lib/python3.8/site-packages/librosa/core/pitch.py:102: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(


10200. FILE PATH: 
 ../data/mp3s/3rMyMv8EjKXoPnaRo2hdJN/Stuntin' Like My Daddy - Street/Birdman, Lil Wayne - Stuntin' Like My Daddy - Street.mp3
10201. FILE PATH: 
 ../data/mp3s/3UUx0JWEJCbKZvhzpruLfe/Letting You Go/Sara Evans - Letting You Go.mp3
10202. FILE PATH: 
 ../data/mp3s/3As1HTUDBTiTDbwnuICy4C/Fisherman's Horizon Piano Collections Version (From Final Fantasy VIII) [For Piano Solo]/Nobuo Uematsu, daigoro789 - #Fisherman's Horizon# Piano Collections Version (From ##Final Fantasy VIII##) [For Piano Solo]#.mp3
10203. FILE PATH: 
 ../data/mp3s/0VSqZ3KStsjcfERGdcWpFO/Les bisous des bisounours/Le Club des Juniors - Les bisous des bisounours.mp3
10204. FILE PATH: 
 ../data/mp3s/7d1ZzOIyACB64aEibmWfH7/Release and Be Free/Kelley Hunt - Release and Be Free.mp3
10205. FILE PATH: 
 ../data/mp3s/4Zhnwb8jvP09aIRAzQMG08/Hospital Beds/Cold War Kids - Hospital Beds.mp3
10206. FILE PATH: 
 ../data/mp3s/2jx8PSP6ncYxZSKqVIoAYa/Cello Concerto in A minor, op. 129 I. Allegro - Nicht zu Schnell/Robert

In [ ]:
# vectorized_df.head()

In [ ]:
# vectorized_df.shape

In [ ]:
# vectorized_df.to_parquet(f"../data/vectorized_mp3s/{output_file_date}_vectorized_audio_data.parquet")

In [ ]:
# vectorized_df.to_csv(f"../data/vectorized_mp3s/{output_file_date}_vectorized_audio_data.csv")

### Test Code

In [ ]:
%%timeit
# Test to see if it could run through 5 songs
tracks,track_ids = [], []
count = 0
for file_path in file_paths:
    if count <= 4:
        # print(file_path)
        print('FILE PATH: \n', f'{file_path} \n')
        x = str(file_path).split('/')
        print(x)
        track_ids.append(x[3])
        array = get_feature(file_path)
        tracks.append(array)
        count+=1
    else:
        break
df = pd.DataFrame(tracks, index = track_ids)
# This implementation also scales everything to 498 dimensions

In [10]:
df.shape

(5, 498)

In [13]:
# assuming 9 seconds per 5 songs
((11573/5)*9)/3600

5.786499999999999

In [22]:
x = get_feature('../data/mp3s/4pFvEWbjBpPUdYRQly0THs/Today Was A Fairytale/Taylor Swift - Today Was A Fairytale.mp3')

In [23]:
x.shape

(498,)

In [29]:
test = pd.DataFrame(x).T
test['track_id']='4pFvEWbjBpPUdYRQly0THs'

In [36]:
test.set_index(test.track_id).drop(columns = 'track_id')

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497
track_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
4pFvEWbjBpPUdYRQly0THs,0.247932,0.296731,0.503741,0.303238,0.281428,0.208925,0.344233,0.558826,0.32845,0.381708,0.255262,0.350181,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,7.753192,26.637911,32.979519,17.698719,14.832067,17.262383,16.535677,23.678122,10.150373,6.05806,6.981291,5.902363,3.228857,2.066222,6.38405,2.992661,5.806331,3.025984,7.050247,2.731785,0.465749,2.9127,8.419796,0.731413,4.010413,5.238231,0.916815,2.880588,5.15386,6.581429,4.607182,0.574148,1.68147,11.119343,4.788751,1.095166,1.402326,13.815983,7.169843,0.562185,0.598196,0.793701,1.214577,1.167978,3.575861,0.601386,0.373227,0.245109,1.130736,0.430002,0.205633,0.19851,1.444831,1.412647,0.815287,1.253845,0.431138,0.479611,0.349126,1.285601,0.530758,0.522415,0.313384,1.220769,0.627867,0.297727,0.208125,0.600484,0.654473,0.268258,0.752402,0.365437,0.54082,0.201796,0.422117,0.238791,0.218321,0.142243,0.228631,0.236479,0.207052,0.21969,0.139043,0.17835,0.152969,0.26196,0.124055,0.096185,0.084213,0.149148,0.145629,0.13509,0.083761,0.122047,0.118967,0.054641,0.060691,0.046777,0.041766,0.036244,0.03507,0.030845,0.032341,0.034754,0.030866,0.031624,0.023949,0.028877,0.025555,0.029167,0.024318,0.031345,0.030118,0.038254,0.037343,0.039811,0.045339,0.044787,0.044473,0.049182,0.048931,0.055415,0.063159,0.053616,0.04905,0.037242,0.01623,0.001063,0.000173,0.000555,0.021913,0.034749,0.115371,0.05533,0.038587,0.299256,0.014102,0.013728,0.098724,0.038773,0.005678,0.007759,0.021868,0.007592,0.002692,0.003139,0.002434,0.001035,0.001123,0.00173,0.013912,0.004592,0.000308,0.000377,0.000